In [1]:
import pandas as pd 
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np 
import visdom
vis = visdom.Visdom()

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

Setting up a new session...


1.4.0
Populating the interactive namespace from numpy and matplotlib


# 1. Data loading & Preprocessing

### 1.1 Data loading

In [2]:
cd /Users/jinsungpark/Desktop/jupyter/Data_river/original02/DA_DG

/Users/jinsungpark/Desktop/jupyter/Data_river/original02/DA_DG


In [3]:
ls #현재경로에 있는 항목 확인

DA_Up_data.xlsx
DA_Up_log.xlsx
DG_Down_data.xlsx
DG_Down_log.xlsx
현풍대암모델-대암덕곡적용_TP(input=10).csv
현풍대암모델-대암덕곡적용_TP(input=14).csv


In [11]:
UpStream_data = pd.read_excel('DA_Up_log.xlsx')
DownStream_data = pd.read_excel('DG_Down_log.xlsx')

In [12]:
print(UpStream_data.columns)
print(DownStream_data.columns)

Index(['Date', 'DA_DO', 'DA_BOD', 'DA_COD', 'DA_SS', 'DA_TN', 'DA_TP',
       'DA_Chl_a', 'DA_Flow', 'HCh__DO', 'HCh__BOD', 'HCh__COD', 'HCh__SS',
       'HCh__TN', 'HCh__TP', 'HCh__Chl_a', 'HCh__Flow', 'UiR_Rain',
       'UiR_Solar', 'DA_Temp', 'HCh__Temp'],
      dtype='object')
Index(['Date', 'DG_DO', 'DG_BOD', 'DG_COD', 'DG_SS', 'DG_TN', 'DG_TP',
       'DG_Chl_a'],
      dtype='object')


In [13]:
#날짜 인덱스화
UpData = UpStream_data.set_index('Date')
DownData = DownStream_data.set_index('Date')

In [14]:
UpData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DA_DO       349 non-null    float64
 1   DA_BOD      349 non-null    float64
 2   DA_COD      349 non-null    float64
 3   DA_SS       349 non-null    float64
 4   DA_TN       349 non-null    float64
 5   DA_TP       349 non-null    float64
 6   DA_Chl_a    349 non-null    float64
 7   DA_Flow     349 non-null    float64
 8   HCh__DO     349 non-null    float64
 9   HCh__BOD    349 non-null    float64
 10  HCh__COD    349 non-null    float64
 11  HCh__SS     349 non-null    float64
 12  HCh__TN     349 non-null    float64
 13  HCh__TP     349 non-null    float64
 14  HCh__Chl_a  349 non-null    float64
 15  HCh__Flow   349 non-null    float64
 16  UiR_Rain    349 non-null    float64
 17  UiR_Solar   349 non-null    float64
 18  DA_Temp     349 non-null    float64
 19  HCh__Temp 

In [15]:
#넣고싶은 상류 항목 컬럼 선택 - TP setting(input:14)
UpData = UpData.iloc[:,[0,2,3,4,5,18,7,8,10,12,13,19,15,16]]
print(UpData.columns)

Index(['DA_DO', 'DA_COD', 'DA_SS', 'DA_TN', 'DA_TP', 'DA_Temp', 'DA_Flow',
       'HCh__DO', 'HCh__COD', 'HCh__TN', 'HCh__TP', 'HCh__Temp', 'HCh__Flow',
       'UiR_Rain'],
      dtype='object')


In [16]:
# #넣고싶은 상류 항목 컬럼 선택 - TN setting
# UpData = UpData.iloc[:,[0,4,5,18,7,8,10,12,13,19,15,16,17]]
# UpData.info()

In [17]:
DownData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 349 entries, 2013-01-07 to 2019-09-30
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DG_DO     349 non-null    float64
 1   DG_BOD    349 non-null    float64
 2   DG_COD    349 non-null    float64
 3   DG_SS     349 non-null    float64
 4   DG_TN     349 non-null    float64
 5   DG_TP     349 non-null    float64
 6   DG_Chl_a  349 non-null    float64
dtypes: float64(7)
memory usage: 21.8 KB


In [18]:
#알고싶은 하류 항목 컬럼 넘버 넣기('Date'항목이 인덱스화 돼서 컬럼 넘버가 -1씩 됨)
Colum = 5
print(DownData.columns[Colum])

DG_TP


### 1.1 Data loading

### 1.2 Data Preprocessing(normalization)

In [19]:
from sklearn.preprocessing import MinMaxScaler

UpScaler = MinMaxScaler() #상류데이터용
DownScaler = MinMaxScaler() #하류데이터용

#나중에 결과를 DeNormalizing 하기 위해 나누어 사용 하였다.

def DeNormalize(Y, Data_name, column_num, Scaler_Type):
    
    data = Data_name
    Scaler = Scaler_Type
    
    _max = Scaler.data_max_[column_num] # 역정규화 하려는 데이터의 컬럼 번호
    _min = Scaler.data_min_[column_num] 
    
    X = Y*(_max-_min) + _min
    
    return X

In [20]:
#데이터 정규화
UpData = pd.DataFrame(UpScaler.fit_transform(UpData), columns=UpData.columns, index=UpData.index)
DownData = pd.DataFrame(DownScaler.fit_transform(DownData), columns=DownData.columns, index=DownData.index)

print(UpData.isna().sum())

DA_DO        0
DA_COD       0
DA_SS        0
DA_TN        0
DA_TP        0
DA_Temp      0
DA_Flow      0
HCh__DO      0
HCh__COD     0
HCh__TN      0
HCh__TP      0
HCh__Temp    0
HCh__Flow    0
UiR_Rain     0
dtype: int64


#2. Data Preparation

In [21]:
class RiverDataset(Dataset):
    def __init__(self, UpData, DownData, x_frames, y_frames, start, end):
        
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = start
        self.end = end

        self.UpData = UpData[start:end]
        self.DownData = DownData[start:end]

    def __len__(self):
        return len(self.UpData) - (self.x_frames + self.y_frames) + 1
    #데이터를 전처리 할때 UpData와 DownData의 길이가 동일해짐(날짜를 동일한것만 추출해야 하므로), 따라서 전체길이는 둘중 하나를 사용

    def __getitem__(self, idx):
        idx += self.x_frames

        X = self.UpData.iloc[idx-self.x_frames:idx].values
        Y = self.DownData.iloc[idx:idx+self.y_frames].values
        
        return X, Y

# 3. Model Define

In [22]:
#input=14
class LSTM_edit(nn.Module):
    
    def __init__(self):
        super(LSTM_edit, self).__init__()
        self.input_dim = 14
        self.hidden_dim = 32
        self.output_dim = 1
        self.num_layers = 2

        self.batch_size = 8
        self.dropout = 0.1
        self.use_bn = False
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers) #
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [23]:
# 정확도 : 예측확률을 100%로 봤을때 MAPE에 따른 오차비율을 빼줌 (100-MAPE) ##RMSE, MAPE 두개로 볼 수 있게
def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [24]:
def RMSE(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    return np.sqrt(mse)

In [25]:
def R2(y_true, y_pred):
    R2_score = r2_score(y_true, y_pred, multioutput='raw_values')
    return R2_score

# 4. Train, Validate, Test

In [26]:
def test_edit(model):
    testloader = DataLoader(testset,
                           batch_size=8,
                           shuffle=False, drop_last=True)
    print('start')
    model.eval()

    bat_siz = 8
    pred = []
    true = []
    pred_results = []
    true_results = []
    test_acc = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X = X.transpose(0, 1).float()
            y_true = y[:, :, Colum].float()
            model.hidden = [hidden for hidden in model.init_hidden()]

            y_pred = model(X)

            pred.append(y_pred)
            true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 ==== #
    for i in range(len(testloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================== #   

    test_acc1 =  RMSE(np.array( true_results), np.array(pred_results))
    test_acc2 =  R2(np.array( true_results), np.array(pred_results))
#     test_acc3 =  (100 - MAPE(np.array( true_results), np.array(pred_results)))
    print('end')
    
    return test_acc1[0], test_acc2[0], pred_results, true_results

# 5. Experiment

# 6. LSTM Run

In [ ]:
print('Current CPU random seed:',torch.initial_seed())
# print('Current CUDA random seed:',torch.cuda.initial_seed())

In [27]:
testset = RiverDataset(UpData, DownData, 2, 1, '2013-01-01', '2017-05-19')

In [28]:
pwd

'/Users/jinsungpark/Desktop/jupyter/Data_river/original02/DA_DG'

In [29]:
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)
modell = LSTM_edit()
modell.load_state_dict(torch.load('/Users/jinsungpark/Desktop/jupyter/Data_river/data04/model_save/현풍-대암_TP(input=14)_보개방이전/lstm06.pt'))

<All keys matched successfully>

In [30]:
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)
RMSE_1, R2_1, Pred_1, True_1 = test_edit(modell)

start
end


In [31]:
test_data = pd.DataFrame()
test_data['test_pred'] = Pred_1
test_data['test_true'] = True_1

acc1 = RMSE(np.array(True_1), np.array(Pred_1))[0]
acc2 = R2(np.array(True_1), np.array(Pred_1))[0]

In [32]:
result = test_data

predict = torch.Tensor(result['test_pred']).view(-1,1)
truth = torch.Tensor(result['test_true']).view(-1,1)
axis = torch.Tensor(range(len(result['test_pred']))).view(-1,1)

Y_axis = torch.cat((predict, truth), -1)
X_axis = torch.cat((axis, axis), -1)

vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result,RMSE[{:2.3f}],R2[{:2.3f}]'
                                           .format(acc1,acc2),
                                           legend=['predict','true'],
                                           showlegend=True,
                                           layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))



'window_385fc9e9999ce0'

In [ ]:
acc2

In [33]:
test_data.to_csv('modell_result_01.csv')

In [ ]:
pwd

In [ ]:
ㅊㅊ